In [ ]:
using DelimitedFiles
using Random
using Statistics
using Printf
include("func.jl");
include("../agentSEIR.jl");

In [ ]:
using PyCall, PyPlot
plt = pyimport("matplotlib.pyplot");

# Read Data

In [ ]:
# Load network data
contact_degree = Int[]
open("../data/montreal/montreal_degree.txt") do io
    for s in eachline(io)
        push!(contact_degree, Int(parse(Float64,s)))
    end
end
contacts = Int[]
open("../data/montreal/montreal_contacts.txt") do io
    for s in eachline(io)
        push!(contacts, Int(parse(Float64,s)))
    end
end

n = length(contact_degree)
end_ = cumsum(contact_degree)
beg_ = vcat(0, end_[1:n-1])
beg_ .+= 1
ll = [contacts[i:j] for (i,j) in zip(beg_, end_)]
end_ = nothing
beg_ = nothing
contacts = nothing
GC.gc()

degree = Int[length(ll[i]) for i in 1:n]
m = Int(sum(degree)/2)
edges = get_edges(ll, n, m)
md = mean(degree)
md2 = mean(degree.^2);

In [ ]:
# Load pre-computed betweenness measures
sp = vec(readdlm("../data/montreal/montreal_shortest_path_betweenness.txt"));
eg = vec(readdlm("../data/montreal/montreal_eigenvector_centrality.txt"));
local02 = vec(readdlm("../data/montreal/montreal_local02_betweenness.txt"));
local04 = vec(readdlm("../data/montreal/montreal_local04_betweenness.txt"));
local10 = vec(readdlm("../data/montreal/montreal_local10_betweenness.txt"));
local25 = vec(readdlm("../data/montreal/montreal_local25_betweenness.txt"));
local50 = vec(readdlm("../data/montreal/montreal_local50_betweenness.txt"));

# Intervention and simulation results

In [ ]:
edge_to_loc = Dict{Tuple{Int,Int},Int}()
for i in 1:length(ll)
    for (loc,j) in enumerate(ll[i])
        edge_to_loc[(i,j)] = loc
    end
end

## Scenario 1a: 85% final size, random initialization

In [ ]:
beta = 0.28

# Verify that beta is set appropriately
t_end = 100
_, sum_S, sum_E, sum_I, sum_R = Main.agentSEIR(ll, days=t_end, beta=beta)
println("Total infection ", sum_R[t_end+1]/n)

In [ ]:
perc = .25;
reduced_weight = .1;

lw_ui = [ones(length(arr)).*(1.0-(1.0-reduced_weight)*perc) for arr in ll];
lw_hd = get_list_of_weights_degree_dist(ll, edge_to_loc, perc, descending=true, weight=reduced_weight);
lw_eg = get_list_of_weights_node_centrality(ll, eg, m, edge_to_loc, perc, weight=reduced_weight);
lw_sp = get_list_of_weights(ll, edges, sp, edge_to_loc, perc, weight=reduced_weight);
lw_lf50 = get_list_of_weights(ll, edges, local50, edge_to_loc, perc, weight=reduced_weight);
lw_lf10 = get_list_of_weights(ll, edges, local10, edge_to_loc, perc, weight=reduced_weight);
lw_lf02 = get_list_of_weights(ll, edges, local02, edge_to_loc, perc, weight=reduced_weight);

##### Sample epidemic curves: random initialization,  25% intervention

In [ ]:
t_end = 60
init_rand_infectious = rand(1:n, Int(ceil(0.001*n)))

_, sum_S_ni, sum_E_ni, sum_I_ni, sum_R_ni = agentSEIR(ll, days=t_end, beta=beta, rand_init=false, init_infectious_nodes=init_rand_infectious);
_, sum_S_ui, sum_E_ui, sum_I_ui, sum_R_ui = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_ui, rand_init=false, init_infectious_nodes=init_rand_infectious);
_, sum_S_hd, sum_E_hd, sum_I_hd, sum_R_hd = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_hd, rand_init=false, init_infectious_nodes=init_rand_infectious);
_, sum_S_eg, sum_E_eg, sum_I_eg, sum_R_eg = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_eg, rand_init=false, init_infectious_nodes=init_rand_infectious);
_, sum_S_sp, sum_E_sp, sum_I_sp, sum_R_sp = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_sp, rand_init=false, init_infectious_nodes=init_rand_infectious);
_, sum_S_lf50, sum_E_lf50, sum_I_lf50, sum_R_lf50 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf50, rand_init=false, init_infectious_nodes=init_rand_infectious);
_, sum_S_lf10, sum_E_lf10, sum_I_lf10, sum_R_lf10 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf10, rand_init=false, init_infectious_nodes=init_rand_infectious);
_, sum_S_lf02, sum_E_lf02, sum_I_lf02, sum_R_lf02 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf02, rand_init=false, init_infectious_nodes=init_rand_infectious);

In [ ]:
t_int = [i for i in 0:t_end]

plt.figure(figsize=(9.3,5.5))
plt.plot(t_int, (sum_E_ni+sum_I_ni)/n, label="NI", linestyle="-", color="dimgray", linewidth=4)
plt.plot(t_int, (sum_E_ui+sum_I_ui)/n, label="UI", linestyle=(0,(5,5)), color="k", linewidth=4)
plt.plot(t_int, (sum_E_eg+sum_I_eg)/n, label="EG", linestyle=(0,(3,1,1,1)), color="tab:brown", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_hd+sum_I_hd)/n, label="HD", linestyle=(0,(3,5,1,5)), color="tab:red", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_sp+sum_I_sp)/n, label="SP", linestyle=(0,(5,1)), color="tab:orange", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf50+sum_I_lf50)/n, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf10+sum_I_lf10)/n, label="LF(1/10)", linestyle=(0,(3,1,1,1,1,1)), color="tab:purple", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf02+sum_I_lf02)/n, label="LF(1/50)", linestyle="dashdot", color="tab:blue", linewidth=4, alpha=1)
leg = plt.legend(fontsize=20, bbox_to_anchor=(.5, 1.26), ncol=4, loc="upper center", handlelength=2)
for i in leg.legendHandles
    i.set_linewidth(4)
end
plt.xticks(size=18)
plt.yticks(size=18)
plt.xlabel("Day", fontsize=22)
plt.ylabel("Active Cases", fontsize=22)
plt.savefig("montreal_curves_randinit_85.tiff", bbox_inches="tight", format="tiff", dpi=400)
plt.show()

### Simulation for interventions that start on day 0

In [ ]:
reduced_weight = 0.1;
t_end = 300;
target_perc = Float64[5.0e-2,.1,.15,.2,.25,.3,.35,.4,.45,.5];

num_scenarios = length(target_perc);
num_trials = 20;

maxCases_ni = zeros(Int, (num_trials,num_scenarios));
maxCases_ui = zeros(Int, (num_trials,num_scenarios));
maxCases_hd = zeros(Int, (num_trials,num_scenarios));
maxCases_eg = zeros(Int, (num_trials,num_scenarios));
maxCases_sp = zeros(Int, (num_trials,num_scenarios));
maxCases_lf50 = zeros(Int, (num_trials,num_scenarios));
maxCases_lf10 = zeros(Int, (num_trials,num_scenarios));
maxCases_lf02 = zeros(Int, (num_trials,num_scenarios));
totalCases_ni = zeros(Int, (num_trials,num_scenarios));
totalCases_ui = zeros(Int, (num_trials,num_scenarios));
totalCases_hd = zeros(Int, (num_trials,num_scenarios));
totalCases_eg = zeros(Int, (num_trials,num_scenarios));
totalCases_sp = zeros(Int, (num_trials,num_scenarios));
totalCases_lf50 = zeros(Int, (num_trials,num_scenarios));
totalCases_lf10 = zeros(Int, (num_trials,num_scenarios));
totalCases_lf02 = zeros(Int, (num_trials,num_scenarios));

for k in 1:num_scenarios
    
    perc = target_perc[k];
    
    lw_ui = [ones(length(arr)).*(1.0-(1.0-reduced_weight)*perc) for arr in ll];
    lw_hd = get_list_of_weights_degree_dist(ll, edge_to_loc, perc, descending=true, weight=reduced_weight);
    lw_eg = get_list_of_weights_node_centrality(ll, eg, m, edge_to_loc, perc, weight=reduced_weight);
    lw_sp = get_list_of_weights(ll, edges, sp, edge_to_loc, perc, weight=reduced_weight);
    lw_lf50 = get_list_of_weights(ll, edges, local50, edge_to_loc, perc, weight=reduced_weight);
    lw_lf10 = get_list_of_weights(ll, edges, local10, edge_to_loc, perc, weight=reduced_weight);
    lw_lf02 = get_list_of_weights(ll, edges, local02, edge_to_loc, perc, weight=reduced_weight);
    
    for trial in 1:num_trials
        
        @printf("perc %.2f \ttrial %d of %d\n", perc, trial, num_trials)
        
        init_rand_infectious = rand(1:n, Int(ceil(0.001*n)))

        _, _, sum_E_ni, sum_I_ni, sum_R_ni = agentSEIR(ll, days=t_end, beta=beta, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_ui, sum_I_ui, sum_R_ui = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_ui, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_hd, sum_I_hd, sum_R_hd = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_hd, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_eg, sum_I_eg, sum_R_eg = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_eg, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _p, sum_E_sp, sum_I_sp, sum_R_sp = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_sp, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_lf50, sum_I_lf50, sum_R_lf50 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf50, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf10, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf02, rand_init=false, init_infectious_nodes=init_rand_infectious);
        
        maxCases_ni[trial,k] = maximum(sum_E_ni + sum_I_ni)
        maxCases_ui[trial,k] = maximum(sum_E_ui + sum_I_ui)
        maxCases_hd[trial,k] = maximum(sum_E_hd + sum_I_hd)
        maxCases_eg[trial,k] = maximum(sum_E_eg + sum_I_eg)
        maxCases_sp[trial,k] = maximum(sum_E_sp + sum_I_sp)
        maxCases_lf50[trial,k] = maximum(sum_E_lf50 + sum_I_lf50)
        maxCases_lf10[trial,k] = maximum(sum_E_lf10 + sum_I_lf10)
        maxCases_lf02[trial,k] = maximum(sum_E_lf02 + sum_I_lf02)   
        totalCases_ni[trial,k] = sum_R_ni[t_end+1]
        totalCases_ui[trial,k] = sum_R_ui[t_end+1]
        totalCases_hd[trial,k] = sum_R_hd[t_end+1]
        totalCases_eg[trial,k] = sum_R_eg[t_end+1]
        totalCases_sp[trial,k] = sum_R_sp[t_end+1]
        totalCases_lf50[trial,k] = sum_R_lf50[t_end+1]
        totalCases_lf10[trial,k] = sum_R_lf10[t_end+1]
        totalCases_lf02[trial,k] = sum_R_lf02[t_end+1]
    end
end

In [ ]:
maxCases_ni_mean = vec(mean(maxCases_ni, dims=1))./n;
maxCases_ui_mean = vec(mean(maxCases_ui, dims=1))./n;
maxCases_hd_mean = vec(mean(maxCases_hd, dims=1))./n;
maxCases_eg_mean = vec(mean(maxCases_eg, dims=1))./n;
maxCases_sp_mean = vec(mean(maxCases_sp, dims=1))./n;
maxCases_lf50_mean = vec(mean(maxCases_lf50, dims=1))./n;
maxCases_lf10_mean = vec(mean(maxCases_lf10, dims=1))./n;
maxCases_lf02_mean = vec(mean(maxCases_lf02, dims=1))./n;
totalCases_ni_mean = vec(mean(totalCases_ni, dims=1))./n;
totalCases_ui_mean = vec(mean(totalCases_ui, dims=1))./n;
totalCases_hd_mean = vec(mean(totalCases_hd, dims=1))./n;
totalCases_eg_mean = vec(mean(totalCases_eg, dims=1))./n;
totalCases_sp_mean = vec(mean(totalCases_sp, dims=1))./n;
totalCases_lf50_mean = vec(mean(totalCases_lf50, dims=1))./n;
totalCases_lf10_mean = vec(mean(totalCases_lf10, dims=1))./n;
totalCases_lf02_mean = vec(mean(totalCases_lf02, dims=1))./n;

plt.figure(figsize=(9.3,5.5))
plt.plot(target_perc, maxCases_ni_mean, label="NI", linestyle="-", color="dimgray", linewidth=6)
plt.plot(target_perc, maxCases_ui_mean, label="UI", linestyle=(0,(5,5)), color="k", linewidth=6)
plt.plot(target_perc, maxCases_hd_mean, label="HD", linestyle=(0,(3,5,1,5)), color="tab:red", linewidth=6)
plt.plot(target_perc, maxCases_eg_mean, label="EG", linestyle=(0,(3,1,1,1)), color="tab:brown", linewidth=6)
plt.plot(target_perc, maxCases_sp_mean, label="SP", linestyle=(0,(5,1)), color="tab:orange", linewidth=6)
plt.plot(target_perc, maxCases_lf50_mean, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, maxCases_lf10_mean, label="LF(1/10)", linestyle=(0,(3,1,1,1,1,1)), color="tab:purple", linewidth=6)
plt.plot(target_perc, maxCases_lf02_mean, label="LF(1/50)", linestyle="dashdot", color="tab:blue", linewidth=6)
leg = plt.legend(fontsize=20, bbox_to_anchor=(.5, 1.26), ncol=4, loc="upper center", handlelength=2)
for i in leg.legendHandles
    i.set_linewidth(4)
end
plt.xticks((.1, .2, .3, .4, .5), ("10%", "20%", "30%", "40%", "50%"), color="k", size=18)
plt.yticks((.2, .3, .4, .5), size=18)
plt.xlabel("Percentage of Targeted Edges", fontsize=22)
plt.ylabel("Epidemic Peak",fontsize=22)
plt.savefig("montreal_epipeak_randinit_85.tiff", bbox_inches="tight", format="tiff", dpi=400)
plt.show()

plt.figure(figsize=(9.3,5.5))
plt.plot(target_perc, totalCases_ni_mean, label="NI", linestyle="-", color="dimgray", linewidth=6)
plt.plot(target_perc, totalCases_ui_mean, label="UI", linestyle=(0,(5,5)), color="k", linewidth=6)
plt.plot(target_perc, totalCases_hd_mean, label="HD", linestyle=(0,(3,5,1,5)), color="tab:red", linewidth=6)
plt.plot(target_perc, totalCases_eg_mean, label="EG", linestyle=(0,(3,1,1,1)), color="tab:brown", linewidth=6)
plt.plot(target_perc, totalCases_sp_mean, label="SP", linestyle=(0,(5,1)), color="tab:orange", linewidth=6)
plt.plot(target_perc, totalCases_lf50_mean, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, totalCases_lf10_mean, label="LF(1/10)", linestyle=(0,(3,1,1,1,1,1)), color="tab:purple", linewidth=6)
plt.plot(target_perc, totalCases_lf02_mean, label="LF(1/50)", linestyle="dashdot", color="tab:blue", linewidth=6)
leg = plt.legend(fontsize=20, bbox_to_anchor=(.5, 1.26), ncol=4, loc="upper center", handlelength=2)
for i in leg.legendHandles
    i.set_linewidth(4)
end
plt.xticks((.1, .2, .3, .4, .5), ("10%", "20%", "30%", "40%", "50%"), color="k", size=18)
plt.yticks(size=18)
plt.xlabel("Percentage of Targeted Edges", fontsize=22)
plt.ylabel("Final Epidemic Size",fontsize=22)
plt.savefig("montreal_episize_randinit_85.tiff", bbox_inches="tight", format="tiff", dpi=400)
plt.show()

## Scenario 1b: 85% final size, cluster initialization

In [ ]:
init_infectious = vec(readdlm("../data/montreal/montreal_initial_cluster.txt", '\t', Int, '\n'));

In [ ]:
beta = 0.28

# Verify that beta is set appropriately
t_end = 100
_, sum_S, sum_E, sum_I, sum_R = Main.agentSEIR(ll, days=t_end, beta=beta, rand_init=false, init_infectious_nodes=init_infectious)
println("Total infection ", sum_R[t_end+1]/n)

In [ ]:
perc = .25;
reduced_weight = .1;

lw_ui = [ones(length(arr)).*(1.0-(1.0-reduced_weight)*perc) for arr in ll];
lw_hd = get_list_of_weights_degree_dist(ll, edge_to_loc, perc, descending=true, weight=reduced_weight);
lw_eg = get_list_of_weights_node_centrality(ll, eg, m, edge_to_loc, perc, weight=reduced_weight);
lw_sp = get_list_of_weights(ll, edges, sp, edge_to_loc, perc, weight=reduced_weight);
lw_lf50 = get_list_of_weights(ll, edges, local50, edge_to_loc, perc, weight=reduced_weight);
lw_lf10 = get_list_of_weights(ll, edges, local10, edge_to_loc, perc, weight=reduced_weight);
lw_lf02 = get_list_of_weights(ll, edges, local02, edge_to_loc, perc, weight=reduced_weight);

##### Sample epidemic curves: cluster initialization, 25% intervention

In [ ]:
t_end = 60

_, sum_S_ni, sum_E_ni, sum_I_ni, sum_R_ni = agentSEIR(ll, days=t_end, beta=beta, rand_init=false, init_infectious_nodes=init_infectious);
_, sum_S_ui, sum_E_ui, sum_I_ui, sum_R_ui = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_ui, rand_init=false, init_infectious_nodes=init_infectious);
_, sum_S_hd, sum_E_hd, sum_I_hd, sum_R_hd = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_hd, rand_init=false, init_infectious_nodes=init_infectious);
_, sum_S_eg, sum_E_eg, sum_I_eg, sum_R_eg = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_eg, rand_init=false, init_infectious_nodes=init_infectious);
_, sum_S_sp, sum_E_sp, sum_I_sp, sum_R_sp = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_sp, rand_init=false, init_infectious_nodes=init_infectious);
_, sum_S_lf50, sum_E_lf50, sum_I_lf50, sum_R_lf50 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf50, rand_init=false, init_infectious_nodes=init_infectious);
_, sum_S_lf10, sum_E_lf10, sum_I_lf10, sum_R_lf10 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf10, rand_init=false, init_infectious_nodes=init_infectious);
_, sum_S_lf02, sum_E_lf02, sum_I_lf02, sum_R_lf02 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf02, rand_init=false, init_infectious_nodes=init_infectious);

In [ ]:
t_int = [i for i in 0:t_end]

plt.figure(figsize=(9.3,5.5))
plt.plot(t_int, (sum_E_ni+sum_I_ni)/n, label="NI", linestyle="-", color="dimgray", linewidth=4)
plt.plot(t_int, (sum_E_ui+sum_I_ui)/n, label="UI", linestyle=(0,(5,5)), color="k", linewidth=4)
plt.plot(t_int, (sum_E_eg+sum_I_eg)/n, label="EG", linestyle=(0,(3,1,1,1)), color="tab:brown", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_hd+sum_I_hd)/n, label="HD", linestyle=(0,(3,5,1,5)), color="tab:red", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_sp+sum_I_sp)/n, label="SP", linestyle=(0,(5,1)), color="tab:orange", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf50+sum_I_lf50)/n, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf10+sum_I_lf10)/n, label="LF(1/10)", linestyle=(0,(3,1,1,1,1,1)), color="tab:purple", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf02+sum_I_lf02)/n, label="LF(1/50)", linestyle="dashdot", color="tab:blue", linewidth=4, alpha=1)
leg = plt.legend(fontsize=20, bbox_to_anchor=(.5, 1.26), ncol=4, loc="upper center", handlelength=2)
for i in leg.legendHandles
    i.set_linewidth(4)
end
plt.xticks(size=18)
plt.yticks(size=18)
plt.xlabel("Day", fontsize=22)
plt.ylabel("Active Cases", fontsize=22)
plt.savefig("montreal_curves_cluster_85.tiff", bbox_inches="tight", format="tiff", dpi=400)
plt.show()

### Simulation for interventions that start on day 0

In [ ]:
reduced_weight = 0.1;
t_end = 300;
target_perc = Float64[5.0e-2,.1,.15,.2,.25,.3,.35,.4,.45,.5];

num_scenarios = length(target_perc);
num_trials = 20;

maxCases_ni = zeros(Int, (num_trials,num_scenarios));
maxCases_ui = zeros(Int, (num_trials,num_scenarios));
maxCases_hd = zeros(Int, (num_trials,num_scenarios));
maxCases_eg = zeros(Int, (num_trials,num_scenarios));
maxCases_sp = zeros(Int, (num_trials,num_scenarios));
maxCases_lf50 = zeros(Int, (num_trials,num_scenarios));
maxCases_lf10 = zeros(Int, (num_trials,num_scenarios));
maxCases_lf02 = zeros(Int, (num_trials,num_scenarios));
totalCases_ni = zeros(Int, (num_trials,num_scenarios));
totalCases_ui = zeros(Int, (num_trials,num_scenarios));
totalCases_hd = zeros(Int, (num_trials,num_scenarios));
totalCases_eg = zeros(Int, (num_trials,num_scenarios));
totalCases_sp = zeros(Int, (num_trials,num_scenarios));
totalCases_lf50 = zeros(Int, (num_trials,num_scenarios));
totalCases_lf10 = zeros(Int, (num_trials,num_scenarios));
totalCases_lf02 = zeros(Int, (num_trials,num_scenarios));

for k in 1:num_scenarios
    
    perc = target_perc[k];
    
    lw_ui = [ones(length(arr)).*(1.0-(1.0-reduced_weight)*perc) for arr in ll];
    lw_hd = get_list_of_weights_degree_dist(ll, edge_to_loc, perc, descending=true, weight=reduced_weight);
    lw_eg = get_list_of_weights_node_centrality(ll, eg, m, edge_to_loc, perc, weight=reduced_weight);
    lw_sp = get_list_of_weights(ll, edges, sp, edge_to_loc, perc, weight=reduced_weight);
    lw_lf50 = get_list_of_weights(ll, edges, local50, edge_to_loc, perc, weight=reduced_weight);
    lw_lf10 = get_list_of_weights(ll, edges, local10, edge_to_loc, perc, weight=reduced_weight);
    lw_lf02 = get_list_of_weights(ll, edges, local02, edge_to_loc, perc, weight=reduced_weight);
    
    for trial in 1:num_trials
        
        @printf("perc %.2f \ttrial %d of %d\n", perc, trial, num_trials)

        _, _, sum_E_ni, sum_I_ni, sum_R_ni = agentSEIR(ll, days=t_end, beta=beta, rand_init=false, init_infectious_nodes=init_infectious);
        _, _, sum_E_ui, sum_I_ui, sum_R_ui = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_ui, rand_init=false, init_infectious_nodes=init_infectious);
        _, _, sum_E_hd, sum_I_hd, sum_R_hd = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_hd, rand_init=false, init_infectious_nodes=init_infectious);
        _, _, sum_E_eg, sum_I_eg, sum_R_eg = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_eg, rand_init=false, init_infectious_nodes=init_infectious);
        _, _p, sum_E_sp, sum_I_sp, sum_R_sp = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_sp, rand_init=false, init_infectious_nodes=init_infectious);
        _, _, sum_E_lf50, sum_I_lf50, sum_R_lf50 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf50, rand_init=false, init_infectious_nodes=init_infectious);
        _, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf10, rand_init=false, init_infectious_nodes=init_infectious);
        _, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf02, rand_init=false, init_infectious_nodes=init_infectious);
        
        maxCases_ni[trial,k] = maximum(sum_E_ni + sum_I_ni)
        maxCases_ui[trial,k] = maximum(sum_E_ui + sum_I_ui)
        maxCases_hd[trial,k] = maximum(sum_E_hd + sum_I_hd)
        maxCases_eg[trial,k] = maximum(sum_E_eg + sum_I_eg)
        maxCases_sp[trial,k] = maximum(sum_E_sp + sum_I_sp)
        maxCases_lf50[trial,k] = maximum(sum_E_lf50 + sum_I_lf50)
        maxCases_lf10[trial,k] = maximum(sum_E_lf10 + sum_I_lf10)
        maxCases_lf02[trial,k] = maximum(sum_E_lf02 + sum_I_lf02)   
        totalCases_ni[trial,k] = sum_R_ni[t_end+1]
        totalCases_ui[trial,k] = sum_R_ui[t_end+1]
        totalCases_hd[trial,k] = sum_R_hd[t_end+1]
        totalCases_eg[trial,k] = sum_R_eg[t_end+1]
        totalCases_sp[trial,k] = sum_R_sp[t_end+1]
        totalCases_lf50[trial,k] = sum_R_lf50[t_end+1]
        totalCases_lf10[trial,k] = sum_R_lf10[t_end+1]
        totalCases_lf02[trial,k] = sum_R_lf02[t_end+1]
    end
end

In [ ]:
maxCases_ni_mean = vec(mean(maxCases_ni, dims=1))./n;
maxCases_ui_mean = vec(mean(maxCases_ui, dims=1))./n;
maxCases_hd_mean = vec(mean(maxCases_hd, dims=1))./n;
maxCases_eg_mean = vec(mean(maxCases_eg, dims=1))./n;
maxCases_sp_mean = vec(mean(maxCases_sp, dims=1))./n;
maxCases_lf50_mean = vec(mean(maxCases_lf50, dims=1))./n;
maxCases_lf10_mean = vec(mean(maxCases_lf10, dims=1))./n;
maxCases_lf02_mean = vec(mean(maxCases_lf02, dims=1))./n;
totalCases_ni_mean = vec(mean(totalCases_ni, dims=1))./n;
totalCases_ui_mean = vec(mean(totalCases_ui, dims=1))./n;
totalCases_hd_mean = vec(mean(totalCases_hd, dims=1))./n;
totalCases_eg_mean = vec(mean(totalCases_eg, dims=1))./n;
totalCases_sp_mean = vec(mean(totalCases_sp, dims=1))./n;
totalCases_lf50_mean = vec(mean(totalCases_lf50, dims=1))./n;
totalCases_lf10_mean = vec(mean(totalCases_lf10, dims=1))./n;
totalCases_lf02_mean = vec(mean(totalCases_lf02, dims=1))./n;

plt.figure(figsize=(9.3,5.5))
plt.plot(target_perc, maxCases_ni_mean, label="NI", linestyle="-", color="dimgray", linewidth=6)
plt.plot(target_perc, maxCases_ui_mean, label="UI", linestyle=(0,(5,5)), color="k", linewidth=6)
plt.plot(target_perc, maxCases_hd_mean, label="HD", linestyle=(0,(3,5,1,5)), color="tab:red", linewidth=6)
plt.plot(target_perc, maxCases_eg_mean, label="EG", linestyle=(0,(3,1,1,1)), color="tab:brown", linewidth=6)
plt.plot(target_perc, maxCases_sp_mean, label="SP", linestyle=(0,(5,1)), color="tab:orange", linewidth=6)
plt.plot(target_perc, maxCases_lf50_mean, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, maxCases_lf10_mean, label="LF(1/10)", linestyle=(0,(3,1,1,1,1,1)), color="tab:purple", linewidth=6)
plt.plot(target_perc, maxCases_lf02_mean, label="LF(1/50)", linestyle="dashdot", color="tab:blue", linewidth=6)
leg = plt.legend(fontsize=20, bbox_to_anchor=(.5, 1.26), ncol=4, loc="upper center", handlelength=2)
for i in leg.legendHandles
    i.set_linewidth(4)
end
plt.xticks((.1, .2, .3, .4, .5), ("10%", "20%", "30%", "40%", "50%"), color="k", size=18)
plt.yticks((.2, .3, .4, .5), size=18)
plt.xlabel("Percentage of Targeted Edges", fontsize=22)
plt.ylabel("Epidemic Peak",fontsize=22)
plt.savefig("montreal_epipeak_cluster_85.tiff", bbox_inches="tight", format="tiff", dpi=400)
plt.show()

plt.figure(figsize=(9.3,5.5))
plt.plot(target_perc, totalCases_ni_mean, label="NI", linestyle="-", color="dimgray", linewidth=6)
plt.plot(target_perc, totalCases_ui_mean, label="UI", linestyle=(0,(5,5)), color="k", linewidth=6)
plt.plot(target_perc, totalCases_hd_mean, label="HD", linestyle=(0,(3,5,1,5)), color="tab:red", linewidth=6)
plt.plot(target_perc, totalCases_eg_mean, label="EG", linestyle=(0,(3,1,1,1)), color="tab:brown", linewidth=6)
plt.plot(target_perc, totalCases_sp_mean, label="SP", linestyle=(0,(5,1)), color="tab:orange", linewidth=6)
plt.plot(target_perc, totalCases_lf50_mean, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, totalCases_lf10_mean, label="LF(1/10)", linestyle=(0,(3,1,1,1,1,1)), color="tab:purple", linewidth=6)
plt.plot(target_perc, totalCases_lf02_mean, label="LF(1/50)", linestyle="dashdot", color="tab:blue", linewidth=6)
leg = plt.legend(fontsize=20, bbox_to_anchor=(.5, 1.26), ncol=4, loc="upper center", handlelength=2)
for i in leg.legendHandles
    i.set_linewidth(4)
end
plt.xticks((.1, .2, .3, .4, .5), ("10%", "20%", "30%", "40%", "50%"), color="k", size=18)
plt.yticks(size=18)
plt.xlabel("Percentage of Targeted Edges", fontsize=22)
plt.ylabel("Final Epidemic Size",fontsize=22)
plt.savefig("montreal_episize_cluster_85.tiff", bbox_inches="tight", format="tiff", dpi=400)
plt.show()

## Scenario 1c: 85% final size, random initialization + delay

In [ ]:
beta = 0.28

# Verify that beta is set appropriately
t_end = 100
_, sum_S, sum_E, sum_I, sum_R = Main.agentSEIR(ll, days=t_end, beta=beta)
println("Total infection ", sum_R[t_end+1]/n)

In [ ]:
perc = .25;
reduced_weight = .1;

lw_ui = [ones(length(arr)).*(1.0-(1.0-reduced_weight)*perc) for arr in ll];
lw_hd = get_list_of_weights_degree_dist(ll, edge_to_loc, perc, descending=true, weight=reduced_weight);
lw_eg = get_list_of_weights_node_centrality(ll, eg, m, edge_to_loc, perc, weight=reduced_weight);
lw_sp = get_list_of_weights(ll, edges, sp, edge_to_loc, perc, weight=reduced_weight);
lw_lf50 = get_list_of_weights(ll, edges, local50, edge_to_loc, perc, weight=reduced_weight);
lw_lf10 = get_list_of_weights(ll, edges, local10, edge_to_loc, perc, weight=reduced_weight);
lw_lf02 = get_list_of_weights(ll, edges, local02, edge_to_loc, perc, weight=reduced_weight);

##### Sample epidemic curves: random initialization, 25% intervention + delay

In [ ]:
t_end = 60
t_delay = 5
t_con = t_end - t_delay

init_state, sum_S, sum_E, sum_I, sum_R = Main.agentSEIR(ll, days=t_delay, beta=beta);
_, _, sum_E_ni, sum_I_ni, sum_R_ni = Main.agentSEIR(ll, days=t_con, beta=beta, rand_init=false, init_state=init_state);
_, _, sum_E_ui, sum_I_ui, sum_R_ui = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=true, lw=lw_ui, rand_init=false, init_state=init_state);
_, _, sum_E_hd, sum_I_hd, sum_R_hd = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=true, lw=lw_hd, rand_init=false, init_state=init_state);
_, _, sum_E_eg, sum_I_eg, sum_R_eg = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=true, lw=lw_eg, rand_init=false, init_state=init_state);
_, _, sum_E_sp, sum_I_sp, sum_R_sp = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=true, lw=lw_sp, rand_init=false, init_state=init_state);
_, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=true, lw=lw_lf10, rand_init=false, init_state=init_state);
_, _, sum_E_lf50, sum_I_lf50, sum_R_lf50 = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=true, lw=lw_lf04, rand_init=false, init_state=init_state);
_, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=true, lw=lw_lf02, rand_init=false, init_state=init_state);

In [ ]:
t_ini = [i for i in 0:t_delay]
t_int = [i for i in t_delay:t_end]

plt.figure(figsize=(9.3,5.5))
plt.plot(t_ini, (sum_E+sum_I)/n, linestyle="-", color="dimgray", linewidth=4)
plt.plot(t_int, (sum_E_ni+sum_I_ni)/n, label="NI", linestyle="-", color="dimgray", linewidth=4)
plt.plot(t_int, (sum_E_ui+sum_I_ui)/n, label="UI", linestyle=(0,(5,5)), color="k", linewidth=4)
plt.plot(t_int, (sum_E_eg+sum_I_eg)/n, label="EG", linestyle=(0,(3,1,1,1)), color="tab:brown", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_hd+sum_I_hd)/n, label="HD", linestyle=(0,(3,5,1,5)), color="tab:red", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_sp+sum_I_sp)/n, label="SP", linestyle=(0,(5,1)), color="tab:orange", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf50+sum_I_lf50)/n, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf10+sum_I_lf10)/n, label="LF(1/10)", linestyle=(0,(3,1,1,1,1,1)), color="tab:purple", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf02+sum_I_lf02)/n, label="LF(1/50)", linestyle="dashdot", color="tab:blue", linewidth=4, alpha=1)
leg = plt.legend(fontsize=20, bbox_to_anchor=(.5, 1.26), ncol=4, loc="upper center", handlelength=2)
for i in leg.legendHandles
    i.set_linewidth(4)
end
plt.xticks(size=18)
plt.yticks(size=18)
plt.xlabel("Day", fontsize=22)
plt.ylabel("Active Cases", fontsize=22)
plt.savefig("montreal_curves_randinit_85_delay5.tiff", bbox_inches="tight", format="tiff", dpi=400)
plt.show()

### Simulation for interventions that start on day 5

In [ ]:
reduced_weight = .1;
target_perc = Float64[5.0e-2,.1,.15,.2,.25,.3,.35,.4,.45,.5];
t_end = 200
t_delay = 5
t_con = t_end - t_delay

num_scenarios = length(target_perc);
num_trials = 50;

maxCases_ni = zeros(Int, (num_trials,num_scenarios));
maxCases_ui = zeros(Int, (num_trials,num_scenarios));
maxCases_hd = zeros(Int, (num_trials,num_scenarios));
maxCases_eg = zeros(Int, (num_trials,num_scenarios));
maxCases_sp = zeros(Int, (num_trials,num_scenarios));
maxCases_lf50 = zeros(Int, (num_trials,num_scenarios));
maxCases_lf10 = zeros(Int, (num_trials,num_scenarios));
maxCases_lf02 = zeros(Int, (num_trials,num_scenarios));
totalCases_ni = zeros(Int, (num_trials,num_scenarios));
totalCases_ui = zeros(Int, (num_trials,num_scenarios));
totalCases_hd = zeros(Int, (num_trials,num_scenarios));
totalCases_eg = zeros(Int, (num_trials,num_scenarios));
totalCases_sp = zeros(Int, (num_trials,num_scenarios));
totalCases_lf50 = zeros(Int, (num_trials,num_scenarios));
totalCases_lf10 = zeros(Int, (num_trials,num_scenarios));
totalCases_lf02 = zeros(Int, (num_trials,num_scenarios));

for k in 1:num_scenarios
    
    perc = target_perc[k];
    
    lw_ui = [ones(length(arr)).*(1.0-(1.0-reduced_weight)*perc) for arr in ll];
    lw_hd = get_list_of_weights_degree_dist(ll, edge_to_loc, perc, descending=true, weight=reduced_weight);
    lw_eg = get_list_of_weights_node_centrality(ll, eg, m, edge_to_loc, perc, weight=reduced_weight);
    lw_sp = get_list_of_weights(ll, edges, sp, edge_to_loc, perc, weight=reduced_weight);
    lw_lf50 = get_list_of_weights(ll, edges, local50, edge_to_loc, perc, weight=reduced_weight);
    lw_lf10 = get_list_of_weights(ll, edges, local10, edge_to_loc, perc, weight=reduced_weight);
    lw_lf02 = get_list_of_weights(ll, edges, local02, edge_to_loc, perc, weight=reduced_weight);
    
    for trial in 1:num_trials
        
        @printf("perc %.2f \ttrial %d of %d\n", perc, trial, num_trials)
        
        init_state, _, _, _, _ = Main.agentSEIR(ll, days=t_delay, beta=beta);
        _, _, sum_E_ni, sum_I_ni, sum_R_ni = Main.agentSEIR(ll, days=t_con, beta=beta, rand_init=false, init_state=init_state);
        _, _, sum_E_ui, sum_I_ui, sum_R_ui = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=true, lw=lw_ui, rand_init=false, init_state=init_state);
        _, _, sum_E_hd, sum_I_hd, sum_R_hd = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=true, lw=lw_hd, rand_init=false, init_state=init_state);
        _, _, sum_E_eg, sum_I_eg, sum_R_eg = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=true, lw=lw_eg, rand_init=false, init_state=init_state);
        _, _, sum_E_sp, sum_I_sp, sum_R_sp = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=true, lw=lw_sp, rand_init=false, init_state=init_state);
        _, _, sum_E_lf50, sum_I_lf50, sum_R_lf50 = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=true, lw=lw_lf50, rand_init=false, init_state=init_state);
        _, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=true, lw=lw_lf10, rand_init=false, init_state=init_state);
        _, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = Main.agentSEIR(ll, days=t_con, beta=beta, weighted=true, lw=lw_lf02, rand_init=false, init_state=init_state);
        
        maxCases_ni[trial,k] = maximum(sum_E_ni + sum_I_ni)
        maxCases_ui[trial,k] = maximum(sum_E_ui + sum_I_ui)
        maxCases_hd[trial,k] = maximum(sum_E_hd + sum_I_hd)
        maxCases_eg[trial,k] = maximum(sum_E_eg + sum_I_eg)
        maxCases_sp[trial,k] = maximum(sum_E_sp + sum_I_sp)
        maxCases_lf50[trial,k] = maximum(sum_E_lf50 + sum_I_lf50)
        maxCases_lf10[trial,k] = maximum(sum_E_lf10 + sum_I_lf10)
        maxCases_lf02[trial,k] = maximum(sum_E_lf02 + sum_I_lf02)   
        totalCases_ni[trial,k] = sum_R_ni[t_con+1]
        totalCases_ui[trial,k] = sum_R_ui[t_con+1]
        totalCases_hd[trial,k] = sum_R_hd[t_con+1]
        totalCases_eg[trial,k] = sum_R_eg[t_con+1]
        totalCases_sp[trial,k] = sum_R_sp[t_con+1]
        totalCases_lf50[trial,k] = sum_R_lf50[t_con+1]
        totalCases_lf10[trial,k] = sum_R_lf10[t_con+1]
        totalCases_lf02[trial,k] = sum_R_lf02[t_con+1]
    end
end

In [ ]:
maxCases_ni_mean = vec(mean(maxCases_ni, dims=1))./n;
maxCases_ui_mean = vec(mean(maxCases_ui, dims=1))./n;
maxCases_hd_mean = vec(mean(maxCases_hd, dims=1))./n;
maxCases_eg_mean = vec(mean(maxCases_eg, dims=1))./n;
maxCases_sp_mean = vec(mean(maxCases_sp, dims=1))./n;
maxCases_lf50_mean = vec(mean(maxCases_lf50, dims=1))./n;
maxCases_lf10_mean = vec(mean(maxCases_lf10, dims=1))./n;
maxCases_lf02_mean = vec(mean(maxCases_lf02, dims=1))./n;
totalCases_ni_mean = vec(mean(totalCases_ni, dims=1))./n;
totalCases_ui_mean = vec(mean(totalCases_ui, dims=1))./n;
totalCases_hd_mean = vec(mean(totalCases_hd, dims=1))./n;
totalCases_eg_mean = vec(mean(totalCases_eg, dims=1))./n;
totalCases_sp_mean = vec(mean(totalCases_sp, dims=1))./n;
totalCases_lf50_mean = vec(mean(totalCases_lf50, dims=1))./n;
totalCases_lf10_mean = vec(mean(totalCases_lf10, dims=1))./n;
totalCases_lf02_mean = vec(mean(totalCases_lf02, dims=1))./n;

plt.figure(figsize=(9.3,5.5))
plt.plot(target_perc, maxCases_ni_mean, label="NI", linestyle="-", color="dimgray", linewidth=6)
plt.plot(target_perc, maxCases_ui_mean, label="UI", linestyle=(0,(5,5)), color="k", linewidth=6)
plt.plot(target_perc, maxCases_hd_mean, label="HD", linestyle=(0,(3,5,1,5)), color="tab:red", linewidth=6)
plt.plot(target_perc, maxCases_eg_mean, label="EG", linestyle=(0,(3,1,1,1)), color="tab:brown", linewidth=6)
plt.plot(target_perc, maxCases_sp_mean, label="SP", linestyle=(0,(5,1)), color="tab:orange", linewidth=6)
plt.plot(target_perc, maxCases_lf50_mean, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, maxCases_lf10_mean, label="LF(1/10)", linestyle=(0,(3,1,1,1,1,1)), color="tab:purple", linewidth=6)
plt.plot(target_perc, maxCases_lf02_mean, label="LF(1/50)", linestyle="dashdot", color="tab:blue", linewidth=6)
leg = plt.legend(fontsize=20, bbox_to_anchor=(.5, 1.26), ncol=4, loc="upper center", handlelength=2)
for i in leg.legendHandles
    i.set_linewidth(4)
end
plt.xticks((.1, .2, .3, .4, .5), ("10%", "20%", "30%", "40%", "50%"), color="k", size=18)
plt.yticks((.2, .3, .4, .5), size=18)
plt.xlabel("Percentage of Targeted Edges", fontsize=22)
plt.ylabel("Epidemic Peak",fontsize=22)
plt.savefig("montreal_epipeak_randinit_85_delay5.tiff", bbox_inches="tight", format="tiff", dpi=400)
plt.show()

plt.figure(figsize=(9.3,5.5))
plt.plot(target_perc, totalCases_ni_mean, label="NI", linestyle="-", color="dimgray", linewidth=6)
plt.plot(target_perc, totalCases_ui_mean, label="UI", linestyle=(0,(5,5)), color="k", linewidth=6)
plt.plot(target_perc, totalCases_hd_mean, label="HD", linestyle=(0,(3,5,1,5)), color="tab:red", linewidth=6)
plt.plot(target_perc, totalCases_eg_mean, label="EG", linestyle=(0,(3,1,1,1)), color="tab:brown", linewidth=6)
plt.plot(target_perc, totalCases_sp_mean, label="SP", linestyle=(0,(5,1)), color="tab:orange", linewidth=6)
plt.plot(target_perc, totalCases_lf50_mean, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, totalCases_lf10_mean, label="LF(1/10)", linestyle=(0,(3,1,1,1,1,1)), color="tab:purple", linewidth=6)
plt.plot(target_perc, totalCases_lf02_mean, label="LF(1/50)", linestyle="dashdot", color="tab:blue", linewidth=6)
leg = plt.legend(fontsize=20, bbox_to_anchor=(.5, 1.26), ncol=4, loc="upper center", handlelength=2)
for i in leg.legendHandles
    i.set_linewidth(4)
end
plt.xticks((.1, .2, .3, .4, .5), ("10%", "20%", "30%", "40%", "50%"), color="k", size=18)
plt.yticks(size=18)
plt.xlabel("Percentage of Targeted Edges", fontsize=22)
plt.ylabel("Final Epidemic Size",fontsize=22)
plt.savefig("montreal_episize_randinit_85_delay5.tiff", bbox_inches="tight", format="tiff", dpi=400)
plt.show()

# Scenario 2: 70% final size, random initialization

In [ ]:
beta = 0.13

# Verify that beta is set appropriately
t_end = 100
_, sum_S, sum_E, sum_I, sum_R = Main.agentSEIR(ll, days=t_end, beta=beta)
println("Total infection ", sum_R[t_end+1]/n)

In [ ]:
perc = .25;
reduced_weight = .1;

lw_ui = [ones(length(arr)).*(1.0-(1.0-reduced_weight)*perc) for arr in ll];
lw_hd = get_list_of_weights_degree_dist(ll, edge_to_loc, perc, descending=true, weight=reduced_weight);
lw_eg = get_list_of_weights_node_centrality(ll, eg, m, edge_to_loc, perc, weight=reduced_weight);
lw_sp = get_list_of_weights(ll, edges, sp, edge_to_loc, perc, weight=reduced_weight);
lw_lf50 = get_list_of_weights(ll, edges, local50, edge_to_loc, perc, weight=reduced_weight);
lw_lf10 = get_list_of_weights(ll, edges, local10, edge_to_loc, perc, weight=reduced_weight);
lw_lf02 = get_list_of_weights(ll, edges, local02, edge_to_loc, perc, weight=reduced_weight);

##### Sample epidemic curves: random initialization, 25% intervention

In [ ]:
t_end = 60
init_rand_infectious = rand(1:n, Int(ceil(0.001*n)))

_, sum_S_ni, sum_E_ni, sum_I_ni, sum_R_ni = agentSEIR(ll, days=t_end, beta=beta, rand_init=false, init_infectious_nodes=init_rand_infectious);
_, sum_S_ui, sum_E_ui, sum_I_ui, sum_R_ui = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_ui, rand_init=false, init_infectious_nodes=init_rand_infectious);
_, sum_S_hd, sum_E_hd, sum_I_hd, sum_R_hd = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_hd, rand_init=false, init_infectious_nodes=init_rand_infectious);
_, sum_S_eg, sum_E_eg, sum_I_eg, sum_R_eg = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_eg, rand_init=false, init_infectious_nodes=init_rand_infectious);
_, sum_S_sp, sum_E_sp, sum_I_sp, sum_R_sp = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_sp, rand_init=false, init_infectious_nodes=init_rand_infectious);
_, sum_S_lf50, sum_E_lf50, sum_I_lf50, sum_R_lf50 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf50, rand_init=false, init_infectious_nodes=init_rand_infectious);
_, sum_S_lf10, sum_E_lf10, sum_I_lf10, sum_R_lf10 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf10, rand_init=false, init_infectious_nodes=init_rand_infectious);
_, sum_S_lf02, sum_E_lf02, sum_I_lf02, sum_R_lf02 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf02, rand_init=false, init_infectious_nodes=init_rand_infectious);

In [ ]:
t_int = [i for i in 0:t_end]

plt.figure(figsize=(9.3,5.5))
plt.plot(t_int, (sum_E_ni+sum_I_ni)/n, label="NI", linestyle="-", color="dimgray", linewidth=4)
plt.plot(t_int, (sum_E_ui+sum_I_ui)/n, label="UI", linestyle=(0,(5,5)), color="k", linewidth=4)
plt.plot(t_int, (sum_E_eg+sum_I_eg)/n, label="EG", linestyle=(0,(3,1,1,1)), color="tab:brown", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_hd+sum_I_hd)/n, label="HD", linestyle=(0,(3,5,1,5)), color="tab:red", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_sp+sum_I_sp)/n, label="SP", linestyle=(0,(5,1)), color="tab:orange", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf50+sum_I_lf50)/n, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf10+sum_I_lf10)/n, label="LF(1/10)", linestyle=(0,(3,1,1,1,1,1)), color="tab:purple", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf02+sum_I_lf02)/n, label="LF(1/50)", linestyle="dashdot", color="tab:blue", linewidth=4, alpha=1)
leg = plt.legend(fontsize=20, bbox_to_anchor=(.5, 1.26), ncol=4, loc="upper center", handlelength=2)
for i in leg.legendHandles
    i.set_linewidth(4)
end
plt.xticks(size=18)
plt.yticks((0,.1,.2,.3),size=18)
plt.xlabel("Day", fontsize=22)
plt.ylabel("Active Cases", fontsize=22)
plt.savefig("montreal_curves_randinit_70.tiff", bbox_inches="tight", format="tiff", dpi=400)
plt.show()

### Simulation for interventions that start on day 0

In [ ]:
reduced_weight = 0.1;
t_end = 300;
target_perc = Float64[5.0e-2,.1,.15,.2,.25,.3,.35,.4,.45,.5];

num_scenarios = length(target_perc);
num_trials = 50;

maxCases_ni = zeros(Int, (num_trials,num_scenarios));
maxCases_ui = zeros(Int, (num_trials,num_scenarios));
maxCases_hd = zeros(Int, (num_trials,num_scenarios));
maxCases_eg = zeros(Int, (num_trials,num_scenarios));
maxCases_sp = zeros(Int, (num_trials,num_scenarios));
maxCases_lf50 = zeros(Int, (num_trials,num_scenarios));
maxCases_lf10 = zeros(Int, (num_trials,num_scenarios));
maxCases_lf02 = zeros(Int, (num_trials,num_scenarios));
totalCases_ni = zeros(Int, (num_trials,num_scenarios));
totalCases_ui = zeros(Int, (num_trials,num_scenarios));
totalCases_hd = zeros(Int, (num_trials,num_scenarios));
totalCases_eg = zeros(Int, (num_trials,num_scenarios));
totalCases_sp = zeros(Int, (num_trials,num_scenarios));
totalCases_lf50 = zeros(Int, (num_trials,num_scenarios));
totalCases_lf10 = zeros(Int, (num_trials,num_scenarios));
totalCases_lf02 = zeros(Int, (num_trials,num_scenarios));

for k in 1:num_scenarios
    
    perc = target_perc[k];
    
    lw_ui = [ones(length(arr)).*(1.0-(1.0-reduced_weight)*perc) for arr in ll];
    lw_hd = get_list_of_weights_degree_dist(ll, edge_to_loc, perc, descending=true, weight=reduced_weight);
    lw_eg = get_list_of_weights_node_centrality(ll, eg, m, edge_to_loc, perc, weight=reduced_weight);
    lw_sp = get_list_of_weights(ll, edges, sp, edge_to_loc, perc, weight=reduced_weight);
    lw_lf50 = get_list_of_weights(ll, edges, local50, edge_to_loc, perc, weight=reduced_weight);
    lw_lf10 = get_list_of_weights(ll, edges, local10, edge_to_loc, perc, weight=reduced_weight);
    lw_lf02 = get_list_of_weights(ll, edges, local02, edge_to_loc, perc, weight=reduced_weight);
    
    for trial in 1:num_trials
        
        @printf("perc %.2f \ttrial %d of %d\n", perc, trial, num_trials)
        
        init_rand_infectious = rand(1:n, Int(ceil(0.001*n)))

        _, _, sum_E_ni, sum_I_ni, sum_R_ni = agentSEIR(ll, days=t_end, beta=beta, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_ui, sum_I_ui, sum_R_ui = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_ui, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_hd, sum_I_hd, sum_R_hd = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_hd, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_eg, sum_I_eg, sum_R_eg = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_eg, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _p, sum_E_sp, sum_I_sp, sum_R_sp = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_sp, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_lf50, sum_I_lf50, sum_R_lf50 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf50, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf10, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf02, rand_init=false, init_infectious_nodes=init_rand_infectious);
        
        maxCases_ni[trial,k] = maximum(sum_E_ni + sum_I_ni)
        maxCases_ui[trial,k] = maximum(sum_E_ui + sum_I_ui)
        maxCases_hd[trial,k] = maximum(sum_E_hd + sum_I_hd)
        maxCases_eg[trial,k] = maximum(sum_E_eg + sum_I_eg)
        maxCases_sp[trial,k] = maximum(sum_E_sp + sum_I_sp)
        maxCases_lf50[trial,k] = maximum(sum_E_lf50 + sum_I_lf50)
        maxCases_lf10[trial,k] = maximum(sum_E_lf10 + sum_I_lf10)
        maxCases_lf02[trial,k] = maximum(sum_E_lf02 + sum_I_lf02)   
        totalCases_ni[trial,k] = sum_R_ni[t_end+1]
        totalCases_ui[trial,k] = sum_R_ui[t_end+1]
        totalCases_hd[trial,k] = sum_R_hd[t_end+1]
        totalCases_eg[trial,k] = sum_R_eg[t_end+1]
        totalCases_sp[trial,k] = sum_R_sp[t_end+1]
        totalCases_lf50[trial,k] = sum_R_lf50[t_end+1]
        totalCases_lf10[trial,k] = sum_R_lf10[t_end+1]
        totalCases_lf02[trial,k] = sum_R_lf02[t_end+1]
    end
end

In [ ]:
maxCases_ni_mean = vec(mean(maxCases_ni, dims=1))./n;
maxCases_ui_mean = vec(mean(maxCases_ui, dims=1))./n;
maxCases_hd_mean = vec(mean(maxCases_hd, dims=1))./n;
maxCases_eg_mean = vec(mean(maxCases_eg, dims=1))./n;
maxCases_sp_mean = vec(mean(maxCases_sp, dims=1))./n;
maxCases_lf50_mean = vec(mean(maxCases_lf50, dims=1))./n;
maxCases_lf10_mean = vec(mean(maxCases_lf10, dims=1))./n;
maxCases_lf02_mean = vec(mean(maxCases_lf02, dims=1))./n;
totalCases_ni_mean = vec(mean(totalCases_ni, dims=1))./n;
totalCases_ui_mean = vec(mean(totalCases_ui, dims=1))./n;
totalCases_hd_mean = vec(mean(totalCases_hd, dims=1))./n;
totalCases_eg_mean = vec(mean(totalCases_eg, dims=1))./n;
totalCases_sp_mean = vec(mean(totalCases_sp, dims=1))./n;
totalCases_lf50_mean = vec(mean(totalCases_lf50, dims=1))./n;
totalCases_lf10_mean = vec(mean(totalCases_lf10, dims=1))./n;
totalCases_lf02_mean = vec(mean(totalCases_lf02, dims=1))./n;

plt.figure(figsize=(9.3,5.5))
plt.plot(target_perc, maxCases_ni_mean, label="NI", linestyle="-", color="dimgray", linewidth=6)
plt.plot(target_perc, maxCases_ui_mean, label="UI", linestyle=(0,(5,5)), color="k", linewidth=6)
plt.plot(target_perc, maxCases_hd_mean, label="HD", linestyle=(0,(3,5,1,5)), color="tab:red", linewidth=6)
plt.plot(target_perc, maxCases_eg_mean, label="EG", linestyle=(0,(3,1,1,1)), color="tab:brown", linewidth=6)
plt.plot(target_perc, maxCases_sp_mean, label="SP", linestyle=(0,(5,1)), color="tab:orange", linewidth=6)
plt.plot(target_perc, maxCases_lf50_mean, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, maxCases_lf10_mean, label="LF(1/10)", linestyle=(0,(3,1,1,1,1,1)), color="tab:purple", linewidth=6)
plt.plot(target_perc, maxCases_lf02_mean, label="LF(1/50)", linestyle="dashdot", color="tab:blue", linewidth=6)
leg = plt.legend(fontsize=20, bbox_to_anchor=(.5, 1.26), ncol=4, loc="upper center", handlelength=2)
for i in leg.legendHandles
    i.set_linewidth(4)
end
plt.xticks((.1, .2, .3, .4, .5), ("10%", "20%", "30%", "40%", "50%"), color="k", size=18)
plt.yticks((.1,.2,.3),size=18)
plt.xlabel("Percentage of Targeted Edges", fontsize=22)
plt.ylabel("Epidemic Peak",fontsize=22)
plt.savefig("montreal_epipeak_randinit_70.tiff", bbox_inches="tight", format="tiff", dpi=400)
plt.show()

plt.figure(figsize=(9.3,5.5))
plt.plot(target_perc, totalCases_ni_mean, label="NI", linestyle="-", color="dimgray", linewidth=6)
plt.plot(target_perc, totalCases_ui_mean, label="UI", linestyle=(0,(5,5)), color="k", linewidth=6)
plt.plot(target_perc, totalCases_hd_mean, label="HD", linestyle=(0,(3,5,1,5)), color="tab:red", linewidth=6)
plt.plot(target_perc, totalCases_eg_mean, label="EG", linestyle=(0,(3,1,1,1)), color="tab:brown", linewidth=6)
plt.plot(target_perc, totalCases_sp_mean, label="SP", linestyle=(0,(5,1)), color="tab:orange", linewidth=6)
plt.plot(target_perc, totalCases_lf50_mean, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, totalCases_lf10_mean, label="LF(1/10)", linestyle=(0,(3,1,1,1,1,1)), color="tab:purple", linewidth=6)
plt.plot(target_perc, totalCases_lf02_mean, label="LF(1/50)", linestyle="dashdot", color="tab:blue", linewidth=6)
leg = plt.legend(fontsize=20, bbox_to_anchor=(.5, 1.26), ncol=4, loc="upper center", handlelength=2)
for i in leg.legendHandles
    i.set_linewidth(4)
end
plt.xticks((.1, .2, .3, .4, .5), ("10%", "20%", "30%", "40%", "50%"), color="k", size=18)
plt.yticks(size=18)
plt.xlabel("Percentage of Targeted Edges", fontsize=22)
plt.ylabel("Final Epidemic Size",fontsize=22)
plt.savefig("montreal_episize_randinit_70.tiff", bbox_inches="tight", format="tiff", dpi=400)
plt.show()

# Scenario 3: 55% final size, random initialization

In [ ]:
beta = 0.07

# Verify that beta is set appropriately
t_end = 100
_, sum_S, sum_E, sum_I, sum_R = Main.agentSEIR(ll, days=t_end, beta=beta)
println("Total infection ", sum_R[t_end+1]/n)

In [ ]:
perc = .25;
reduced_weight = .1;

lw_ui = [ones(length(arr)).*(1.0-(1.0-reduced_weight)*perc) for arr in ll];
lw_hd = get_list_of_weights_degree_dist(ll, edge_to_loc, perc, descending=true, weight=reduced_weight);
lw_eg = get_list_of_weights_node_centrality(ll, eg, m, edge_to_loc, perc, weight=reduced_weight);
lw_sp = get_list_of_weights(ll, edges, sp, edge_to_loc, perc, weight=reduced_weight);
lw_lf50 = get_list_of_weights(ll, edges, local50, edge_to_loc, perc, weight=reduced_weight);
lw_lf10 = get_list_of_weights(ll, edges, local10, edge_to_loc, perc, weight=reduced_weight);
lw_lf02 = get_list_of_weights(ll, edges, local02, edge_to_loc, perc, weight=reduced_weight);

##### Sample epidemic curve: random initialization,  25% intervention

In [ ]:
t_end = 60
init_rand_infectious = rand(1:n, Int(ceil(0.001*n)))

_, sum_S_ni, sum_E_ni, sum_I_ni, sum_R_ni = agentSEIR(ll, days=t_end, beta=beta, rand_init=false, init_infectious_nodes=init_rand_infectious);
_, sum_S_ui, sum_E_ui, sum_I_ui, sum_R_ui = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_ui, rand_init=false, init_infectious_nodes=init_rand_infectious);
_, sum_S_hd, sum_E_hd, sum_I_hd, sum_R_hd = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_hd, rand_init=false, init_infectious_nodes=init_rand_infectious);
_, sum_S_eg, sum_E_eg, sum_I_eg, sum_R_eg = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_eg, rand_init=false, init_infectious_nodes=init_rand_infectious);
_, sum_S_sp, sum_E_sp, sum_I_sp, sum_R_sp = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_sp, rand_init=false, init_infectious_nodes=init_rand_infectious);
_, sum_S_lf50, sum_E_lf50, sum_I_lf50, sum_R_lf50 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf50, rand_init=false, init_infectious_nodes=init_rand_infectious);
_, sum_S_lf10, sum_E_lf10, sum_I_lf10, sum_R_lf10 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf10, rand_init=false, init_infectious_nodes=init_rand_infectious);
_, sum_S_lf02, sum_E_lf02, sum_I_lf02, sum_R_lf02 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf02, rand_init=false, init_infectious_nodes=init_rand_infectious);

In [ ]:
t_int = [i for i in 0:t_end]

plt.figure(figsize=(9.3,5.5))
plt.plot(t_int, (sum_E_ni+sum_I_ni)/n, label="NI", linestyle="-", color="dimgray", linewidth=4)
plt.plot(t_int, (sum_E_ui+sum_I_ui)/n, label="UI", linestyle=(0,(5,5)), color="k", linewidth=4)
plt.plot(t_int, (sum_E_eg+sum_I_eg)/n, label="EG", linestyle=(0,(3,1,1,1)), color="tab:brown", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_hd+sum_I_hd)/n, label="HD", linestyle=(0,(3,5,1,5)), color="tab:red", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_sp+sum_I_sp)/n, label="SP", linestyle=(0,(5,1)), color="tab:orange", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf50+sum_I_lf50)/n, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf10+sum_I_lf10)/n, label="LF(1/10)", linestyle=(0,(3,1,1,1,1,1)), color="tab:purple", linewidth=4, alpha=1)
plt.plot(t_int, (sum_E_lf02+sum_I_lf02)/n, label="LF(1/50)", linestyle="dashdot", color="tab:blue", linewidth=4, alpha=1)
leg = plt.legend(fontsize=20, bbox_to_anchor=(.5, 1.26), ncol=4, loc="upper center", handlelength=2)
for i in leg.legendHandles
    i.set_linewidth(4)
end
plt.xticks(size=18)
plt.yticks((0,.1,.2),size=18)
plt.xlabel("Day", fontsize=22)
plt.ylabel("Active Cases", fontsize=22)
plt.savefig("montreal_curves_randinit_55.tiff", bbox_inches="tight", format="tiff", dpi=400)
plt.show()

### Simulation for interventions that start on day 0

In [ ]:
reduced_weight = 0.1;
t_end = 400;
target_perc = Float64[5.0e-2,.1,.15,.2,.25,.3,.35,.4,.45,.5];

num_scenarios = length(target_perc);
num_trials = 50;

maxCases_ni = zeros(Int, (num_trials,num_scenarios));
maxCases_ui = zeros(Int, (num_trials,num_scenarios));
maxCases_hd = zeros(Int, (num_trials,num_scenarios));
maxCases_eg = zeros(Int, (num_trials,num_scenarios));
maxCases_sp = zeros(Int, (num_trials,num_scenarios));
maxCases_lf50 = zeros(Int, (num_trials,num_scenarios));
maxCases_lf10 = zeros(Int, (num_trials,num_scenarios));
maxCases_lf02 = zeros(Int, (num_trials,num_scenarios));
totalCases_ni = zeros(Int, (num_trials,num_scenarios));
totalCases_ui = zeros(Int, (num_trials,num_scenarios));
totalCases_hd = zeros(Int, (num_trials,num_scenarios));
totalCases_eg = zeros(Int, (num_trials,num_scenarios));
totalCases_sp = zeros(Int, (num_trials,num_scenarios));
totalCases_lf50 = zeros(Int, (num_trials,num_scenarios));
totalCases_lf10 = zeros(Int, (num_trials,num_scenarios));
totalCases_lf02 = zeros(Int, (num_trials,num_scenarios));

for k in 1:num_scenarios
    
    perc = target_perc[k];
    
    lw_ui = [ones(length(arr)).*(1.0-(1.0-reduced_weight)*perc) for arr in ll];
    lw_hd = get_list_of_weights_degree_dist(ll, edge_to_loc, perc, descending=true, weight=reduced_weight);
    lw_eg = get_list_of_weights_node_centrality(ll, eg, m, edge_to_loc, perc, weight=reduced_weight);
    lw_sp = get_list_of_weights(ll, edges, sp, edge_to_loc, perc, weight=reduced_weight);
    lw_lf50 = get_list_of_weights(ll, edges, local50, edge_to_loc, perc, weight=reduced_weight);
    lw_lf10 = get_list_of_weights(ll, edges, local10, edge_to_loc, perc, weight=reduced_weight);
    lw_lf02 = get_list_of_weights(ll, edges, local02, edge_to_loc, perc, weight=reduced_weight);
    
    for trial in 1:num_trials
        
        @printf("perc %.2f \ttrial %d of %d\n", perc, trial, num_trials)
        
        init_rand_infectious = rand(1:n, Int(ceil(0.001*n)))

        _, _, sum_E_ni, sum_I_ni, sum_R_ni = agentSEIR(ll, days=t_end, beta=beta, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_ui, sum_I_ui, sum_R_ui = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_ui, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_hd, sum_I_hd, sum_R_hd = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_hd, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_eg, sum_I_eg, sum_R_eg = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_eg, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _p, sum_E_sp, sum_I_sp, sum_R_sp = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_sp, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_lf50, sum_I_lf50, sum_R_lf50 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf50, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf10, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf02, rand_init=false, init_infectious_nodes=init_rand_infectious);
        
        maxCases_ni[trial,k] = maximum(sum_E_ni + sum_I_ni)
        maxCases_ui[trial,k] = maximum(sum_E_ui + sum_I_ui)
        maxCases_hd[trial,k] = maximum(sum_E_hd + sum_I_hd)
        maxCases_eg[trial,k] = maximum(sum_E_eg + sum_I_eg)
        maxCases_sp[trial,k] = maximum(sum_E_sp + sum_I_sp)
        maxCases_lf50[trial,k] = maximum(sum_E_lf50 + sum_I_lf50)
        maxCases_lf10[trial,k] = maximum(sum_E_lf10 + sum_I_lf10)
        maxCases_lf02[trial,k] = maximum(sum_E_lf02 + sum_I_lf02)   
        totalCases_ni[trial,k] = sum_R_ni[t_end+1]
        totalCases_ui[trial,k] = sum_R_ui[t_end+1]
        totalCases_hd[trial,k] = sum_R_hd[t_end+1]
        totalCases_eg[trial,k] = sum_R_eg[t_end+1]
        totalCases_sp[trial,k] = sum_R_sp[t_end+1]
        totalCases_lf50[trial,k] = sum_R_lf50[t_end+1]
        totalCases_lf10[trial,k] = sum_R_lf10[t_end+1]
        totalCases_lf02[trial,k] = sum_R_lf02[t_end+1]
    end
end

In [ ]:
maxCases_ni_mean = vec(mean(maxCases_ni, dims=1))./n;
maxCases_ui_mean = vec(mean(maxCases_ui, dims=1))./n;
maxCases_hd_mean = vec(mean(maxCases_hd, dims=1))./n;
maxCases_eg_mean = vec(mean(maxCases_eg, dims=1))./n;
maxCases_sp_mean = vec(mean(maxCases_sp, dims=1))./n;
maxCases_lf50_mean = vec(mean(maxCases_lf50, dims=1))./n;
maxCases_lf10_mean = vec(mean(maxCases_lf10, dims=1))./n;
maxCases_lf02_mean = vec(mean(maxCases_lf02, dims=1))./n;
totalCases_ni_mean = vec(mean(totalCases_ni, dims=1))./n;
totalCases_ui_mean = vec(mean(totalCases_ui, dims=1))./n;
totalCases_hd_mean = vec(mean(totalCases_hd, dims=1))./n;
totalCases_eg_mean = vec(mean(totalCases_eg, dims=1))./n;
totalCases_sp_mean = vec(mean(totalCases_sp, dims=1))./n;
totalCases_lf50_mean = vec(mean(totalCases_lf50, dims=1))./n;
totalCases_lf10_mean = vec(mean(totalCases_lf10, dims=1))./n;
totalCases_lf02_mean = vec(mean(totalCases_lf02, dims=1))./n;

plt.figure(figsize=(9.3,5.5))
plt.plot(target_perc, maxCases_ni_mean, label="NI", linestyle="-", color="dimgray", linewidth=6)
plt.plot(target_perc, maxCases_ui_mean, label="UI", linestyle=(0,(5,5)), color="k", linewidth=6)
plt.plot(target_perc, maxCases_hd_mean, label="HD", linestyle=(0,(3,5,1,5)), color="tab:red", linewidth=6)
plt.plot(target_perc, maxCases_eg_mean, label="EG", linestyle=(0,(3,1,1,1)), color="tab:brown", linewidth=6)
plt.plot(target_perc, maxCases_sp_mean, label="SP", linestyle=(0,(5,1)), color="tab:orange", linewidth=6)
plt.plot(target_perc, maxCases_lf50_mean, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, maxCases_lf10_mean, label="LF(1/10)", linestyle=(0,(3,1,1,1,1,1)), color="tab:purple", linewidth=6)
plt.plot(target_perc, maxCases_lf02_mean, label="LF(1/50)", linestyle="dashdot", color="tab:blue", linewidth=6)
leg = plt.legend(fontsize=20, bbox_to_anchor=(.5, 1.26), ncol=4, loc="upper center", handlelength=2)
for i in leg.legendHandles
    i.set_linewidth(4)
end
plt.xticks((.1, .2, .3, .4, .5), ("10%", "20%", "30%", "40%", "50%"), color="k", size=18)
plt.yticks((.1,.2),size=18)
plt.xlabel("Percentage of Targeted Edges", fontsize=22)
plt.ylabel("Epidemic Peak",fontsize=22)
plt.savefig("montreal_epipeak_randinit_55.tiff", bbox_inches="tight", format="tiff", dpi=400)
plt.show()

plt.figure(figsize=(9.3,5.5))
plt.plot(target_perc, totalCases_ni_mean, label="NI", linestyle="-", color="dimgray", linewidth=6)
plt.plot(target_perc, totalCases_ui_mean, label="UI", linestyle=(0,(5,5)), color="k", linewidth=6)
plt.plot(target_perc, totalCases_hd_mean, label="HD", linestyle=(0,(3,5,1,5)), color="tab:red", linewidth=6)
plt.plot(target_perc, totalCases_eg_mean, label="EG", linestyle=(0,(3,1,1,1)), color="tab:brown", linewidth=6)
plt.plot(target_perc, totalCases_sp_mean, label="SP", linestyle=(0,(5,1)), color="tab:orange", linewidth=6)
plt.plot(target_perc, totalCases_lf50_mean, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, totalCases_lf10_mean, label="LF(1/10)", linestyle=(0,(3,1,1,1,1,1)), color="tab:purple", linewidth=6)
plt.plot(target_perc, totalCases_lf02_mean, label="LF(1/50)", linestyle="dashdot", color="tab:blue", linewidth=6)
leg = plt.legend(fontsize=20, bbox_to_anchor=(.5, 1.26), ncol=4, loc="upper center", handlelength=2)
for i in leg.legendHandles
    i.set_linewidth(4)
end
plt.xticks((.1, .2, .3, .4, .5), ("10%", "20%", "30%", "40%", "50%"), color="k", size=18)
plt.yticks((.2,.3,.4,.5),size=18)
plt.xlabel("Percentage of Targeted Edges", fontsize=22)
plt.ylabel("Final Epidemic Size",fontsize=22)
plt.savefig("montreal_episize_randinit_55.tiff", bbox_inches="tight", format="tiff", dpi=400)
plt.show()

# Node immunization

In [ ]:
sp_n = edge_to_node_betweenness(edges, sp, n);
local02_n = edge_to_node_betweenness(edges, local02, n);
local10_n = edge_to_node_betweenness(edges, local10, n);
local50_n = edge_to_node_betweenness(edges, local50, n);

In [ ]:
reduced_weight = 0.0;
t_end = 300;
target_perc = Float64[5.0e-2,.1,.15,.2,.25,.3];

num_scenarios = length(target_perc);
num_trials = 50;

maxCases_ni = zeros(Int, (num_trials,num_scenarios));
maxCases_ui = zeros(Int, (num_trials,num_scenarios));
maxCases_hd = zeros(Int, (num_trials,num_scenarios));
maxCases_eg = zeros(Int, (num_trials,num_scenarios));
maxCases_sp = zeros(Int, (num_trials,num_scenarios));
maxCases_lf50 = zeros(Int, (num_trials,num_scenarios));
maxCases_lf10 = zeros(Int, (num_trials,num_scenarios));
maxCases_lf02 = zeros(Int, (num_trials,num_scenarios));
totalCases_ni = zeros(Int, (num_trials,num_scenarios));
totalCases_ui = zeros(Int, (num_trials,num_scenarios));
totalCases_hd = zeros(Int, (num_trials,num_scenarios));
totalCases_eg = zeros(Int, (num_trials,num_scenarios));
totalCases_sp = zeros(Int, (num_trials,num_scenarios));
totalCases_lf50 = zeros(Int, (num_trials,num_scenarios));
totalCases_lf10 = zeros(Int, (num_trials,num_scenarios));
totalCases_lf02 = zeros(Int, (num_trials,num_scenarios));

for k in 1:num_scenarios
    
    perc = target_perc[k];
    
    lw_ui = [ones(length(arr)).*(1.0-(1.0-reduced_weight)*perc) for arr in ll];
    lw_hd = get_list_of_weights_degree_dist_target_nodes(ll, edge_to_loc, perc, descending=true, weight=reduced_weight);
    lw_eg = get_list_of_weights_node_centrality_target_nodes(ll, eg, edge_to_loc, perc, weight=reduced_weight);
    lw_sp = get_list_of_weights_node_centrality_target_nodes(ll, sp_n, edge_to_loc, perc, weight=reduced_weight);
    lw_lf50 = get_list_of_weights_node_centrality_target_nodes(ll, local50_n, edge_to_loc, perc, weight=reduced_weight);
    lw_lf10 = get_list_of_weights_node_centrality_target_nodes(ll, local10_n, edge_to_loc, perc, weight=reduced_weight);
    lw_lf02 = get_list_of_weights_node_centrality_target_nodes(ll, local02_n, edge_to_loc, perc, weight=reduced_weight);
    
    for trial in 1:num_trials
        
        @printf("perc %.2f \ttrial %d of %d \n", perc, trial, num_trials)
        
        init_rand_infectious = rand(1:n, Int(ceil(0.001*n)))

        _, _, sum_E_ni, sum_I_ni, sum_R_ni = agentSEIR(ll, days=t_end, beta=beta, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_ui, sum_I_ui, sum_R_ui = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_ui, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_hd, sum_I_hd, sum_R_hd = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_hd, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_eg, sum_I_eg, sum_R_eg = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_eg, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _p, sum_E_sp, sum_I_sp, sum_R_sp = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_sp, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_lf50, sum_I_lf50, sum_R_lf50 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf50, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf10, rand_init=false, init_infectious_nodes=init_rand_infectious);
        _, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = agentSEIR(ll, days=t_end, beta=beta, weighted=true, lw=lw_lf02, rand_init=false, init_infectious_nodes=init_rand_infectious);
        
        maxCases_ni[trial,k] = maximum(sum_E_ni + sum_I_ni)
        maxCases_ui[trial,k] = maximum(sum_E_ui + sum_I_ui)
        maxCases_hd[trial,k] = maximum(sum_E_hd + sum_I_hd)
        maxCases_eg[trial,k] = maximum(sum_E_eg + sum_I_eg)
        maxCases_sp[trial,k] = maximum(sum_E_sp + sum_I_sp)
        maxCases_lf50[trial,k] = maximum(sum_E_lf50 + sum_I_lf50)
        maxCases_lf10[trial,k] = maximum(sum_E_lf10 + sum_I_lf10)
        maxCases_lf02[trial,k] = maximum(sum_E_lf02 + sum_I_lf02)   
        totalCases_ni[trial,k] = sum_R_ni[t_end+1]
        totalCases_ui[trial,k] = sum_R_ui[t_end+1]
        totalCases_hd[trial,k] = sum_R_hd[t_end+1]
        totalCases_eg[trial,k] = sum_R_eg[t_end+1]
        totalCases_sp[trial,k] = sum_R_sp[t_end+1]
        totalCases_lf50[trial,k] = sum_R_lf50[t_end+1]
        totalCases_lf10[trial,k] = sum_R_lf10[t_end+1]
        totalCases_lf02[trial,k] = sum_R_lf02[t_end+1]
    end
end

In [ ]:
maxCases_ni_mean = vec(mean(maxCases_ni, dims=1))./n;
maxCases_ui_mean = vec(mean(maxCases_ui, dims=1))./n;
maxCases_hd_mean = vec(mean(maxCases_hd, dims=1))./n;
maxCases_eg_mean = vec(mean(maxCases_eg, dims=1))./n;
maxCases_sp_mean = vec(mean(maxCases_sp, dims=1))./n;
maxCases_lf50_mean = vec(mean(maxCases_lf50, dims=1))./n;
maxCases_lf10_mean = vec(mean(maxCases_lf10, dims=1))./n;
maxCases_lf02_mean = vec(mean(maxCases_lf02, dims=1))./n;
totalCases_ni_mean = vec(mean(totalCases_ni, dims=1))./n;
totalCases_ui_mean = vec(mean(totalCases_ui, dims=1))./n;
totalCases_hd_mean = vec(mean(totalCases_hd, dims=1))./n;
totalCases_eg_mean = vec(mean(totalCases_eg, dims=1))./n;
totalCases_sp_mean = vec(mean(totalCases_sp, dims=1))./n;
totalCases_lf50_mean = vec(mean(totalCases_lf50, dims=1))./n;
totalCases_lf10_mean = vec(mean(totalCases_lf10, dims=1))./n;
totalCases_lf02_mean = vec(mean(totalCases_lf02, dims=1))./n;

plt.figure(figsize=(9.3,5.5))
plt.plot(target_perc, maxCases_ni_mean, label="NI", linestyle="-", color="dimgray", linewidth=6)
plt.plot(target_perc, maxCases_ui_mean, label="UI", linestyle=(0,(5,5)), color="k", linewidth=6)
plt.plot(target_perc, maxCases_hd_mean, label="HD", linestyle=(0,(3,5,1,5)), color="tab:red", linewidth=6)
plt.plot(target_perc, maxCases_eg_mean, label="EG", linestyle=(0,(3,1,1,1)), color="tab:brown", linewidth=6)
plt.plot(target_perc, maxCases_sp_mean, label="SP", linestyle=(0,(5,1)), color="tab:orange", linewidth=6)
plt.plot(target_perc, maxCases_lf50_mean, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, maxCases_lf10_mean, label="LF(1/10)", linestyle=(0,(3,1,1,1,1,1)), color="tab:purple", linewidth=6)
plt.plot(target_perc, maxCases_lf02_mean, label="LF(1/50)", linestyle="dashdot", color="tab:blue", linewidth=6)
leg = plt.legend(fontsize=20, bbox_to_anchor=(.5, 1.26), ncol=4, loc="upper center", handlelength=2)
for i in leg.legendHandles
    i.set_linewidth(4)
end
plt.xticks((5.0e-2,.1,.15,.2,.25,.3), ("5%","10%","15%","20%","25%","30%"), color="k", size=18)
plt.yticks(size=18)
plt.xlabel("Percentage of Targeted Nodes", fontsize=22)
plt.ylabel("Epidemic Peak",fontsize=22)
plt.savefig("montreal_epipeak_randinit_85_immu.png", bbox_inches="tight", format="png", dpi=400)
plt.show()

plt.figure(figsize=(9.3,5.5))
plt.plot(target_perc, totalCases_ni_mean, label="NI", linestyle="-", color="dimgray", linewidth=6)
plt.plot(target_perc, totalCases_ui_mean, label="UI", linestyle=(0,(5,5)), color="k", linewidth=6)
plt.plot(target_perc, totalCases_hd_mean, label="HD", linestyle=(0,(3,5,1,5)), color="tab:red", linewidth=6)
plt.plot(target_perc, totalCases_eg_mean, label="EG", linestyle=(0,(3,1,1,1)), color="tab:brown", linewidth=6)
plt.plot(target_perc, totalCases_sp_mean, label="SP", linestyle=(0,(5,1)), color="tab:orange", linewidth=6)
plt.plot(target_perc, totalCases_lf50_mean, label="LF(1/2)", linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, totalCases_lf10_mean, label="LF(1/10)", linestyle=(0,(3,1,1,1,1,1)), color="tab:purple", linewidth=6)
plt.plot(target_perc, totalCases_lf02_mean, label="LF(1/50)", linestyle="dashdot", color="tab:blue", linewidth=6)
leg = plt.legend(fontsize=20, bbox_to_anchor=(.5, 1.26), ncol=4, loc="upper center", handlelength=2)
for i in leg.legendHandles
    i.set_linewidth(4)
end
plt.xticks((5.0e-2,.1,.15,.2,.25,.3), ("5%","10%","15%","20%","25%","30%"), color="k", size=18)
plt.yticks(size=18)
plt.xlabel("Percentage of Targeted Nodes", fontsize=22)
plt.ylabel("Final Epidemic Size",fontsize=22)
plt.savefig("montreal_episize_randinit_85_immu.png", bbox_inches="tight", format="png", dpi=400)
plt.show()